In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import matplotlib.pyplot as plt
import pickle
import numpy as np
import random
import argparse
import secrets
from certificate.run_simulations import run_experiments, generate_arm_means
from certificate.utils import delete_duplicate_results
import json 
import sys
import os

In [17]:
is_jupyter = 'ipykernel' in sys.modules

In [86]:
if is_jupyter: 
    seed        = 43
    trials = 100
    n_arms = 10
    max_pulls_per_arm = 50
    first_stage_pulls_per_arm = 25
    arm_distribution = 'uniform'
    out_folder = "uniform"
    arm_parameters=  {} # {'alpha': 50, 'beta': 50, 'diff_mean_1': 0.05, 'diff_std_1': 0.01,'diff_mean_2': 0.01, 'diff_std_2': 0.001}
    delta = 0.1
    run_all_k = True
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', help='Random Seed', type=int, default=42)
    parser.add_argument('--trials', help='Trials', type=int, default=25)
    parser.add_argument('--n_arms',         '-N', help='Number of arms', type=int, default=10)
    parser.add_argument('--max_pulls_per_arm',        help='Maximum pulls per arm', type=int, default=10)
    parser.add_argument('--first_stage_pulls_per_arm',          help='Number of first stage pulls ', type=int, default=4)
    parser.add_argument('--arm_distribution',          help='Distribution of arms', type=str, default='uniform')
    parser.add_argument('--run_all_k',        help='Maximum pulls per arm', action='store_true')
    parser.add_argument('--delta',        help='Maximum pulls per arm', type=float, default=0.1)
    parser.add_argument('--alpha',        help='Maximum pulls per arm', type=float, default=2)
    parser.add_argument('--beta',        help='Maximum pulls per arm', type=float, default=2)
    parser.add_argument('--diff_mean_1',        help='Maximum pulls per arm', type=float, default=2)
    parser.add_argument('--diff_std_1',        help='Maximum pulls per arm', type=float, default=2)
    parser.add_argument('--diff_mean_2',        help='Maximum pulls per arm', type=float, default=2)
    parser.add_argument('--diff_std_2',        help='Maximum pulls per arm', type=float, default=2)
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='policy_comparison')

    args = parser.parse_args()

    seed = args.seed
    n_arms = args.n_arms
    max_pulls_per_arm = args.max_pulls_per_arm 
    first_stage_pulls_per_arm = args.first_stage_pulls_per_arm
    arm_distribution = args.arm_distribution
    out_folder = args.out_folder
    delta = args.delta 
    alpha = args.alpha 
    beta = args.beta 
    trials = args.trials 
    diff_mean_1 = args.diff_mean_1 
    diff_std_1 = args.diff_std_1 
    diff_mean_2 = args.diff_mean_2 
    diff_std_2 = args.diff_std_2
    arm_parameters = {'alpha': alpha, 'beta': beta, 'diff_mean_1': diff_mean_1, 'diff_mean_2': diff_mean_2, 'diff_std_1': diff_std_1, 'diff_std_2': diff_std_2}
    run_all_k = args.run_all_k

save_name = secrets.token_hex(4)  

In [87]:
random.seed(seed)
np.random.seed(seed)

## Run Policies

In [88]:
arm_means = generate_arm_means(arm_distribution,arm_parameters,n_arms)

In [89]:
experiment_config = {
    'number_arms': n_arms, 
    'sample_size': max_pulls_per_arm*n_arms, 
    'first_stage_size': first_stage_pulls_per_arm*n_arms, 
    'distribution': arm_means, 
    'arm_distribution': arm_distribution, 
    'random_seed': seed+1, 
    'delta': delta,
    'run_all_k': run_all_k, 
    'reward_parameters': arm_parameters, 
    'true_value': np.max(arm_means)
}

In [90]:
all_results = []

for i in range(trials):
    experiment_config['random_seed'] = seed+i
    results = run_experiments(experiment_config)
    all_results.append(results)

In [91]:
aggregate_results = {}
aggregate_results['parameters'] = experiment_config
aggregate_results['parameters']['seed'] = seed 

for method in all_results[0]:
    aggregate_results[method] = {}
    aggregate_results[method]['certificate'] = [max(i[method]['certificate']) for i in all_results]
    aggregate_results[method]['certificate_width'] = [i[method]['certificate_width'] for i in all_results]


In [94]:
for i in aggregate_results:
    if 'certificate' in aggregate_results[i]:
        print(i,np.mean(aggregate_results[i]['certificate']))

omniscient 0.7774390195278785
dominant 0.7590657259876146
sample_split 0.7565952839652458
sample_split_total 0.7643542838379979
two_stage_thompson 0.7676557200905567
two_stage_successive_elimination 0.7078474478008941
ucb 0.7958302814914915
successive_elimination 0.7362139673134714
k_1 0.733938595755849
k_2 0.7483494817562382
k_3 0.736316021662907
k_4 0.7254062237697622
k_5 0.7078572870614853
k_6 0.6964571132023571
k_7 0.6835984578932103
k_8 0.6751557377060249
k_9 0.6593787502305095
k_10 0.6534033973710652
random 0.700345017873978
one_stage 0.7162572870614854


## Write Data

In [33]:
save_path = "{}/{}.json".format(out_folder,save_name)

In [34]:
delete_duplicate_results(out_folder,"",aggregate_results)

In [35]:
if is_jupyter:
    dirname = os.path.abspath('')
    filename = os.path.join(dirname, '../../results/'+save_path)
else:
    dirname = os.path.dirname(os.path.abspath(__file__))
    filename = os.path.join(dirname, '../../results/'+save_path)
json.dump(aggregate_results,open(filename,'w'))